# Sales Data Example

Window functions are commonly used together with sales data. In this notebook we will be using a data set called "Watson Sales Product Sample Data" which was downloaded from https://www.ibm.com/communities/analytics/watson-analytics-blog/sales-products-sample-data/

In [2]:
import pyspark.sql.functions as f

from pyspark.sql.window import Window
from pyspark.sql import SparkSession

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","4G") \
        .getOrCreate()

spark

/opt/anaconda3/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 09:20:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1 Watson Sales Product Sample Data

First we load the data, which is provided as a single CSV file, which again is well supported by Apache Spark

In [3]:
basedir = "s3://dimajix-training/data"
#basedir = "/dimajix/data"

In [4]:
data = spark.read.csv(basedir + "/watson-sales-products/WA_Sales_Products_2012-14.csv", header=True, inferSchema=True)

### Inspect schema

Since we used the existing header information and also let Spark infer appropriate data types, let us inspect the schema now.


In [5]:
data.printSchema()

root
 |-- Retailer country: string (nullable = true)
 |-- Order method type: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Product type: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Gross margin: double (nullable = true)



### Preaggregate data

Since we are not interested in all details, we preaggregate the data into the following columns:
* Retailer country
* Retailer type
* Product line
* Quarter

In [6]:
aggregated_data = data.groupBy(
    "Retailer country",
    "Retailer type",
    "Product line",
    "Quarter"
).agg(
    f.sum("Revenue").alias("Revenue"),
    f.sum("Quantity").alias("Quantity")
)

aggregated_data.printSchema()

root
 |-- Retailer country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)



# 2 Find Difference to Average

In the first example, we try to find the difference of the revenue of each quarter to the average revenue for each retailer country and retailer type over all quarters. This can be done either using a grouped aggregated followed by a join or by using window functions.

## 2.1 Self Join

Just for the sake of completeness, let us start with the aggragetion and join approach. It will turn out later that this is much more complicated than using a window function, but nevertheless we implement this approach such that we can compare both approaches.

### Step 1: Extract year and quarter

Technically the first step is not required, but in order to provide some meaningful sorting, we extract the quarter (Q1, Q2, Q3 and Q4) and the year from the incoming column `Quarter`. Otherwise sorting wouldn't work, since that column is formatted as `'Q'q YYYY` which doesn't provide a chronologically ordering if sorted alphabetically.

In [7]:
extended_data = aggregated_data.select(
    f.col("*"),
    f.substring(aggregated_data["Quarter"],1,2).alias("q"),
    f.substring(aggregated_data["Quarter"],3,8).alias("y")
)

# Inspect the schema (you could also peek at the first 10 records instead)
extended_data.printSchema()

root
 |-- Retailer country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- q: string (nullable = true)
 |-- y: string (nullable = true)



### Step 2: Calculate average revenue

Now we calculate the average revenue per `Retailer Country`, `Retailer type` and `Product line`. Do NOT use a window function, perform a simple grouped aggregation instead (`groupBy(...).agg(..)`) is your friend).

In [9]:
avg_data = extended_data.groupBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    ).agg(
        f.avg("Revenue").alias("avg_revenue")
    )

# Inspect the schema (you could also peek at the first 10 records instead)
avg_data.printSchema()

root
 |-- Retailer country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- avg_revenue: double (nullable = true)



### Step 3: Join and calculate

Now we join the average revenue with the original data set, such that we can calculate the difference of the revenue and the average revenue. To do so, we need to join the two DataFrames `extended_data` and `avg_data` on the relevant columns `Retailer country`, `Retailer type` and `Product line`. This then allows us to compare the revenue within each quarter with the average revenue (both sides partitioned by these three dimensions used for joining).

In [10]:
# Join extended_data with avg_data on the columns "Retailer country", "Retailer type", "Product line"
joined_data = extended_data.join(avg_data, ["Retailer country", "Retailer type", "Product line"])

# Select all columns from "extended_data" and add a new column to "joined_data", which contains the difference 
# beteween the current revenue from "extended_data" and the average revenue from "avg_data"
result = joined_data.select(
        extended_data["*"],
        (extended_data["Revenue"] - avg_data["avg_revenue"]).alias("revenue_delta")
    )

# Finally sort the result by "Retailer Country", "Retailer Type", "Product line", "y", "q", then drop the two helper columns "q" and "y"
sorted_result = result \
    .orderBy("Retailer Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

sorted_result.limit(10).toPandas()

,Retailer country,Retailer type,Product line,Quarter,Revenue,Quantity,revenue_delta
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,-6.546596e+05
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,-4.130929e+05
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,-5.312361e+05
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,-3.612573e+05
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,-1.109529e+05
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,1.110154e+06
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,6.809099e+05
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,8.065727e+04
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,1.334871e+05
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,1.171662e+06


## 2.2 Better use Window Functions

Now let us perform the very same analysis, but using windowed aggregation instead of aggregation and joining. A *window* aggregates groups of records, but this grouping and aggregation will be performed (conceptionally) individually for every input record and the result will be attached to each input record. Therefore a windowed aggregation works like a normal aggregation followed by a join.

In Spark we always need to specify how this aggregation window is to be constructed. It always has up to three components:
* Partitioning - controls which records will be considered for each window
* Sorting - sorts all records in a window
* Range - controls how many records in the sorted list should be aggregated

### Aggregation functions
After the window has been created, you can use any conventional aggregation function like `sum`, `avg` etc. In addition Spark also provides some special window functions which make use of the ordering (which is not available in normal aggregations). The most important window aggregation functions are:
* `rank()`
* `dense_rank()`
* `row_number()`
* `lag(column, n)` and `lead(column, n)`


### Step 1: Extract year and quarter

Technically the first step is not required, but in order to provide some meaningful sorting, we extract the quarter (Q1, Q2, Q3 and Q4) and the year from the incoming column Quarter. Otherwise sorting wouldn't work, since that column is formatted as 'Q'q YYYY which doesn't provide a chronologically ordering if sorted alphabetically.

In [11]:
extended_data = aggregated_data.select(
    f.col("*"),
    f.substring(aggregated_data["Quarter"],1,2).alias("q"),
    f.substring(aggregated_data["Quarter"],3,8).alias("y")
)

### Step 2: Define window

This time we use a windowed aggregation to calculate the average price. As the first step we need to construct a *window*. In this case it contains the following ingredients:
* A definition of partitions (i.e. which rows should be averages together)
* A definition of the window size in rows (i.e. which rows within each partition should take part for each average)

In [16]:
# Define a window
avg_window = Window\
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing) \
    .partitionBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    )

### Step 3: Perform analysis

Now we want to conduct the simple analysis as follows: We calculate the average revenue per "Retailer country", "Retailer type" and "Product line". This effectively removes the "Quarter" from the list of dimensions. The average should be provided as a new column `avg_revenue`. Then we simply subtract this average revenue from each Revenue and store the result again in a new column `revenue_diff`.

In [19]:
# Add the following two columns to the DataFrame "extended_data":
#  * avg_revenue - this should contain the average revenue within each window defined above
#  * revenue_diff - this should contain the difference between the original Revenue and the new "avg_revenue"
result = extended_data \
    .withColumn("avg_revenue", f.avg(extended_data["Revenue"]).over(avg_window)) \
    .withColumn("revenue_diff", f.col("Revenue") - f.col("avg_revenue"))

# Sort result for nicer output
sorted_result = result \
    .orderBy("Retailer Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

sorted_result.limit(10).toPandas()

,Retailer country,Retailer type,Product line,Quarter,Revenue,Quantity,avg_revenue,revenue_diff
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,1.435296e+06,-6.546596e+05
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,1.435296e+06,-4.130929e+05
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,1.435296e+06,-5.312361e+05
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,1.435296e+06,-3.612573e+05
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,1.435296e+06,-1.109529e+05
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,1.435296e+06,1.110154e+06
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,1.435296e+06,6.809099e+05
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,1.435296e+06,8.065727e+04
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,1.435296e+06,1.334871e+05
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,1.435296e+06,1.171662e+06


In [116]:
result.explain(True)

== Parsed Logical Plan ==
'Project [*, ('Revenue - 'avg_revenue) AS revenue_diff#11239]
+- Project [Retailer country#7476, Retailer type#7478, Product line#7479, Quarter#7483, Revenue#11064, Quantity#11066L, q#11218, y#11219, avg_revenue#11229]
   +- Project [Retailer country#7476, Retailer type#7478, Product line#7479, Quarter#7483, Revenue#11064, Quantity#11066L, q#11218, y#11219, avg_revenue#11229, avg_revenue#11229]
      +- Window [avg(Revenue#11064) windowspecdefinition(Retailer country#7476, Retailer type#7478, Product line#7479, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS avg_revenue#11229], [Retailer country#7476, Retailer type#7478, Product line#7479]
         +- Project [Retailer country#7476, Retailer type#7478, Product line#7479, Quarter#7483, Revenue#11064, Quantity#11066L, q#11218, y#11219]
            +- Project [Retailer country#7476, Retailer type#7478, Product line#7479, Quarter#7483, Revenue#11064, Quantity#11066L, substring(Quar

# 3 Best Quarter

Another interesting question would be, which quarter was the best one in each country for each retailer type and product line. This would be already much harder to do with a join, since the join key would probably need to contain the maximum revenue, which is a double (never join on floating point values, it might not work).

## 3.1 Using Window Functions

### Step 2: Perform analysis

Again we need to define a window, and within each window partition we want to sort the rows by the `Revenue` column and add the sorted position as a new column. This then allows us to trivially simply select the top most row in each window, which contains the best revenue. 

This time the window again needs to be partitioned by the dimensions `Retailer country`, `Retailer type` and `Product line`. In order to identified the best quarter, we also need to sort all entries within each window by `Revenue`, such that we can easily pick the top most revenue.

In [21]:
# Define a ranking window. This is defined by
#   * partition by "Retailer country", "Retailer type" and "Product line"
#   * sorted by "Revenue"
rank_window = Window\
    .orderBy(extended_data["Revenue"].desc())\
    .partitionBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    )

### Step 3: Perform analysis

By using this window, we can easily perform the analysis be calculating the position of each record within its window by using the `row_number` function and then select the top most record by filtering the row number to be 1.

In [22]:
# Add a new column "rank" by using the "row_number" window function together with the window defined above. 
ranked_data = extended_data.select(
        f.col("*"),
        f.row_number().over(rank_window).alias("rank")
    )

# Pick the top entry of every window by filtering on the row number. Only keep records with rank = 1
result = ranked_data.filter(ranked_data["rank"] == 1)

# Sort result, just to improve output. Also remove helper columns "q", "y" and "rank"
sorted_result = result \
    .orderBy("Retailer Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y", "rank")

sorted_result.limit(10).toPandas()

,Retailer country,Retailer type,Product line,Quarter,Revenue,Quantity
0,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056
1,Australia,Department Store,Golf Equipment,Q2 2014,677267.68,4916
2,Australia,Department Store,Outdoor Protection,Q4 2012,78711.97,13393
3,Australia,Department Store,Personal Accessories,Q2 2014,819106.85,18057
4,Australia,Direct Marketing,Camping Equipment,Q1 2013,588239.09,16345
5,Australia,Direct Marketing,Golf Equipment,Q1 2014,24586.64,275
6,Australia,Direct Marketing,Outdoor Protection,Q3 2013,49510.36,7455
7,Australia,Direct Marketing,Personal Accessories,Q3 2013,160893.21,4529
8,Australia,Equipment Rental Store,Camping Equipment,Q3 2013,187676.43,3438
9,Australia,Equipment Rental Store,Golf Equipment,Q3 2013,107589.55,192


# 4 Difference between Quarters

Another common example where windowing will greatly simplify processing is accessing different rows in a single query. This cannot be done in Spark without using some trick, since Spark normally processes all rows independently. In a simple `select` you can access any number of columns, but you only have access to a single row.

As an example, we'd like to calculate the difference in revenue of two consecutive quarters. Obviously we need to access the revenue of two quarters to calulcate the difference. Again we use two different approaches, the first using a `join` operation and the second using a windowed aggregation.

## 4.1 Self Join

The first approach will join the data set to itself, such that two different quarters of the same retailer country, retailer type and product type are put together into a single row. Then a simple subtraction will provide the result.

### Step 1: Calculate previous quarter

As a first step, we need to create a small helper function for calculating the previous quarter from a given quarter using the provided format `Qq YYYY`. With this function we can generate the join key required for joining the same dataset on the previous quarter.

We will write a small Python UDF to perform the desired operation.

In [23]:
def prev_quarter(quarter):
    q = int(quarter[1:2])
    y = int(quarter[3:8])
    
    prev_q = q - 1
    if (prev_q <= 0):
        prev_y = y - 1
        prev_q = 4
    else:
        prev_y = y
    
    return "Q" + str(prev_q) + " " + str(prev_y)
    
print(prev_quarter("Q1 2012"))
print(prev_quarter("Q4 2012"))

Q4 2011
Q3 2012


In [27]:
prev_quarter_udf = f.udf(prev_quarter, 'string')

Now we apply the `prev_quarter` UDF to the data set to create a new column containing the previous quarter.

In [28]:
extended_data = aggregated_data.withColumn("prev_quarter", prev_quarter_udf(aggregated_data["Quarter"]))

extended_data.printSchema()

root
 |-- Retailer country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- prev_quarter: string (nullable = true)



### Step 2: Join current and previous Quarter

Now we need to join the current quarter with the last quarter using the newly created column `prev_quarter`

In [30]:
joined_data = extended_data.alias("current") \
        .join(extended_data.alias("prev"),
            (f.col("current.Quarter") == f.col("prev.prev_quarter")) &
            (f.col("current.Retailer country") == f.col("prev.Retailer country")) &
            (f.col("current.Retailer type") == f.col("prev.Retailer type")) &
            (f.col("current.Product Line") == f.col("prev.Product Line")),
        how="left"
    )

joined_data.printSchema()

root
 |-- Retailer country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- prev_quarter: string (nullable = true)
 |-- Retailer country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- prev_quarter: string (nullable = true)



Note that most columns are present twice now, but by using the data frame aliases `current` and `prev` we still can distinguish between the two original sources. We need that capability in the next step.

### Step 3: Calculate difference

Now that we have the current revenue and the previous revenue joined together in a single data frame, we finally can now calculate the difference and keep only the columns from the `current` data frame.

In [31]:
joined_data.select(
        f.col("current.*"),
        (f.col("current.Revenue") - f.col("prev.Revenue")).alias("revenue_delta")
    )

result.limit(10).toPandas()

,Retailer country,Retailer type,Product line,Quarter,Revenue,Quantity,q,y,rank
0,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,Q2,2014,1
1,Australia,Department Store,Golf Equipment,Q2 2014,677267.68,4916,Q2,2014,1
2,Australia,Department Store,Outdoor Protection,Q4 2012,78711.97,13393,Q4,2012,1
3,Australia,Department Store,Personal Accessories,Q2 2014,819106.85,18057,Q2,2014,1
4,Australia,Direct Marketing,Camping Equipment,Q1 2013,588239.09,16345,Q1,2013,1
5,Australia,Direct Marketing,Golf Equipment,Q1 2014,24586.64,275,Q1,2014,1
6,Australia,Direct Marketing,Outdoor Protection,Q3 2013,49510.36,7455,Q3,2013,1
7,Australia,Direct Marketing,Personal Accessories,Q3 2013,160893.21,4529,Q3,2013,1
8,Australia,Equipment Rental Store,Camping Equipment,Q3 2013,187676.43,3438,Q3,2013,1
9,Australia,Equipment Rental Store,Golf Equipment,Q3 2013,107589.55,192,Q3,2013,1


## 4.2 Use Window Functions

Now that we saw how to solve the problem with a join (and a UDF for calculating the previous quarter), let us get to a different approach using a windowed aggregation. 

In [32]:
extended_data = aggregated_data.select(
    f.col("*"),
    f.substring(aggregated_data["Quarter"],1,2).alias("q"),
    f.substring(aggregated_data["Quarter"],3,8).alias("y")
)

### Step 1: Define Window

What we essentially want to do is to access values from *different rows* for calculating the difference between quarters. So what we need is something like follows:
* Create window per retailer country, retailer type and product line
* Sort by quarter
* Pick previous row

The last step is the interesting one. This is done by using the `lag` window aggregation function which allows you to access some preceeding record within the window. Note that the window actually has to contain exactly one record, otherwise you'll get an error by Spark.

In [33]:
# Define a window with the following properties:
#   * partitioned by "Retailer country", "Retailer type" and "Product line"
#   * orderedBy "y" and "q"
#   * rows between -1 and -1 (just pick the previous row in the window)
prev_window = Window \
    .orderBy(extended_data["y"].asc(),extended_data["q"].asc())\
    .rowsBetween(-1, -1) \
    .partitionBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    )

### Step 2: Perform analysis

Now we can use the window in the following simple select statement:

In [35]:
# Add a new column "revenue_delta" to the DataFrame extended_data, which contains the difference between the current revenue and the revenue from the previous quarter.
# Use the "lag" function with the window defined above to access the previous revenue within the DataFrame
result = extended_data.withColumn("revenue_delta", extended_data["Revenue"] - f.lag(extended_data["Revenue"], 1).over(prev_window))

# Sort and tidy up the DataFrame
sorted_result = result \
    .orderBy("Retailer Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

sorted_result.limit(10).toPandas()

,Retailer country,Retailer type,Product line,Quarter,Revenue,Quantity,revenue_delta
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,NaN
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,241566.66
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,-118143.25
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,169978.82
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,250304.39
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,1221107.22
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,-429244.43
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,-600252.59
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,52829.86
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,1038174.71


# 5 Putting it all together

Of course you can also use different window aggregations with different windows in a single query as follows:

In [37]:
rank_window = Window\
    .orderBy(extended_data["Revenue"].desc())\
    .partitionBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    )
avg_window = Window\
    .orderBy(extended_data["Revenue"].desc())\
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing) \
    .partitionBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    )

prev_window = Window \
    .orderBy(extended_data["y"].asc(),extended_data["q"].asc())\
    .rowsBetween(-1, -1) \
    .partitionBy(
        "Retailer country",
        "Retailer type",
        "Product line"
    )

result = extended_data.select(
        f.col("*"),
        f.row_number().over(rank_window).alias("rank"),
        f.avg(extended_data["Revenue"]).over(avg_window).alias("avg_revenue"),
        (extended_data["Revenue"] - f.lag(extended_data["Revenue"], 1).over(prev_window)).alias("revenue_delta")
    )

sorted_result = result\
    .orderBy("Retailer Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

result.limit(10).toPandas()

,Retailer country,Retailer type,Product line,Quarter,Revenue,Quantity,q,y,rank,avg_revenue,revenue_delta
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,Q1,2012,10,1.435296e+06,NaN
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,Q2,2012,8,1.435296e+06,241566.66
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,Q3,2012,9,1.435296e+06,-118143.25
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,Q4,2012,7,1.435296e+06,169978.82
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,Q1,2013,6,1.435296e+06,250304.39
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,Q2,2013,2,1.435296e+06,1221107.22
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,Q3,2013,3,1.435296e+06,-429244.43
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,Q4,2013,5,1.435296e+06,-600252.59
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,Q1,2014,4,1.435296e+06,52829.86
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,Q2,2014,1,1.435296e+06,1038174.71
